In [183]:
import pandas as pd
import os
import numpy as np
import re
from collections import Counter
import matplotlib.pyplot as plt 
import seaborn as sb
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests 


# GETTING THE DATAFRAMES I'M GOING TO WORK WITH #

1. Download the base dataframe

In [184]:
df = pd.read_csv("../data/Dataset Global Fashion Brands Brand Equity Ranking Growth Rate  COO ROO 2001-2021.csv")
df.sample(5)

,BrandName,BrandOriginCountry,BrandOriginRegion,BrandSector,BrandSubSector,Rank2001,Rank2002,Rank2003,Rank2004,Rank2005,...,GrowthRate2012,GrowthRate2013,GrowthRate2014,GrowthRate2015,GrowthRate2016,GrowthRate2017,GrowthRate2018,GrowthRate2019,GrowthRate2020,GrowthRate2021
13,Benetton,Italy,Europe,Fashion,Apparel,100.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,Tiffany,U.S.A.,America,Fashion,Luxury,73.0,72.0,70.0,75.0,81.0,...,14.70,5.45,9.12,6.23,-8.64,-6.37,4.60,-5.44,-6.92,10.43
21,Burberry,U.K.,Europe,Fashion,Luxury,NaN,NaN,NaN,NaN,NaN,...,16.35,19.51,7.80,4.99,-8.70,-4.23,-2.84,4.33,-7.61,8.03
22,Nike,U.S.A.,America,Fashion,Sportswear,34.0,35.0,33.0,31.0,30.0,...,4.12,12.95,16.33,16.08,8.51,7.94,11.47,7.49,6.21,23.70
0,Louis Vuitton,France,Europe,Fashion,Luxury,38.0,41.0,45.0,44.0,18.0,...,1.75,5.58,-9.40,-1.34,7.86,-4.50,22.83,14.46,-1.56,15.91


2. Web scrapping for 2nd dataframe

''' The next comments you'll find are unsuccessful tries to webscraping an url. It didn't work cause they're calling themselfs, tried to do it for a long time and fer found a way 
to download the csv directly so all the next steps until reading the next csv didn't serve but I didn't want to 
delete it cause I spent 3 hours trying to scrap the website.'''

In [185]:
'''url = 'https://www.brandwatch.com/brandwatch-index/luxury-fashion'
res = requests.get(url, timeout=5)
html = res.content
soup = BeautifulSoup(html, 'html.parser')'''

"url = 'https://www.brandwatch.com/brandwatch-index/luxury-fashion'\nres = requests.get(url, timeout=5)\nhtml = res.content\nsoup = BeautifulSoup(html, 'html.parser')"

In [186]:
'''def get_columns(soup):
    index = soup.find("div", {"class": "index-table__row__pos"})
    ind = index.find_all("h5")
    ind_processed = [i.getText() for i in ind]
    return ind_processed'''
    

'def get_columns(soup):\n    index = soup.find("div", {"class": "index-table__row__pos"})\n    ind = index.find_all("h5")\n    ind_processed = [i.getText() for i in ind]\n    return ind_processed'

In [187]:
'''get_columns(soup)'''

'get_columns(soup)'

In [188]:
'''ds = soup.find_all("li", {"class": "lindex-table__row index-table__row--body js-c-si-row"})
ds'''

'ds = soup.find_all("li", {"class": "lindex-table__row index-table__row--body js-c-si-row"})\nds'

In [189]:
'''url = "https://www.brandwatch.com/wp-content/uploads/2020/01/LuxFashion-Q42019.csv?rev=1698422964802"
res = requests.get(url)
res'''

'url = "https://www.brandwatch.com/wp-content/uploads/2020/01/LuxFashion-Q42019.csv?rev=1698422964802"\nres = requests.get(url)\nres'

In [190]:
'''res.content'''

'res.content'

In [191]:
'''import pandas as pd
url = 'https://www.brandwatch.com/brandwatch-index/luxury-fashion'
res = requests.get(url)
soup = BeautifulSoup(res.content, "html.parser")
soup'''


'import pandas as pd\nurl = \'https://www.brandwatch.com/brandwatch-index/luxury-fashion\'\nres = requests.get(url)\nsoup = BeautifulSoup(res.content, "html.parser")\nsoup'

In [192]:
'''ds = soup.find_all("li")
ds[20]'''

'ds = soup.find_all("li")\nds[20]'

2. 1 Finding another way to get the needed data.

    In this case downloaded as csv and imported with pandas

In [193]:
df_2= pd.read_csv("../data/LuxFashion-Q42019.csv")
df_2.sample(5)

,Brand,Social Visibility,Web Visibility,Sentiment,Growth,Search,Total
1,Gucci,100.00,100.00,53.61,23.56,85.14,362.31
4,Coach,9.29,56.64,64.26,0.00,41.18,171.37
29,Lanvin,1.23,2.40,19.21,0.00,1.26,24.10
11,Michael Kors,5.91,17.59,58.40,0.00,49.79,131.69
14,Ralph Lauren,8.39,26.28,37.34,0.00,47.83,119.84


3. Get the 3rd Dataframe by webscrapping

In [194]:
url = 'https://fashionunited.com/i/top200'
res = requests.get(url, timeout=5)
html = res.content
soup = BeautifulSoup(html, 'html.parser')

In [195]:
''' Function to get html content from a web with bs4. '''

def download_html(url):
    res = requests.get(url, timeout=5)
    html = res.content
    return BeautifulSoup(html, 'html.parser')

In [196]:
url = 'https://fashionunited.com/i/top200'
soup = download_html(url)

In [197]:
'''Function to get a table and transform it into a dataframe from an already requested link.'''

def get_table_web(soup):
    table =  soup.find("table")
    rows = table.find_all('tr')
    data = []
    for row in rows:
        cols = row.find_all('td')
        cols = [col.text for col in cols]
        data.append(cols)
    ds = pd.DataFrame(data, columns=['Index','Rank', 'Company', 'Mcap $', 'Type'])
    return ds.drop(['Index'], axis=1)

In [198]:
'''Calling of the function'''

df_3 = get_table_web(soup)
df_3.sample(5)

,Rank,Company,Mcap $,Type
13,13,Rolex SA,$29.84 b,Private
85,85,Hanesbrands Inc.,$2.51 b,Public
9,9,Chanel,$57.46 b,Private
31,31,Pentland Brands,$8.12 b,Private
113,113,Neiman Marcus,$1.03 b,Private


4. Export the last dataframe to csv.

In [199]:
df_3.to_csv(f"../data/100_fashion_money.csv")

5. Get a list of participating brand from the 3 more important fashion weeks: NYFW, Milan FW, Paris FW

         Milan Fashion week info extraction. Participating brands.

In [200]:
url_2 = 'https://www.wallpaper.com/fashion-beauty/milan-fashion-week-aw-2023-highlights'

In [201]:
''' Reusing the by calling it'''

soup = download_html(url_2)

In [202]:
ds =  soup.find("div", {"id": "article-body"})
titles = ds.find_all("h2")
brands = [i.getText() for i in titles]
FWM = [i for i in brands if "Week" not in i]
FWM


['Giorgio Armani',
 'Bottega Veneta',
 'Dolce & Gabbana',
 'MSGM',
 'Ferragamo',
 'Ferrari',
 'Jil Sander',
 'Andreadamo',
 'Gucci',
 'Sportmax',
 'Tod’s',
 'Blumarine',
 'Emporio Armani',
 'Prada',
 'Max Mara',
 'Etro',
 'No. 21',
 'Fendi',
 'Diesel']

    '''Now, it's time to encapsulate the code. I've been doing it and deleting the process to get there, but maybe it's a good idea to leave it. '''

In [203]:
''' Function to get titles from an article body'''

def titles_article(soup):
    ds =  soup.find("div", {"id": "article-body"})
    titles = ds.find_all("h2")
    brands = [i.getText() for i in titles]
    return [i for i in brands if "Week" not in i]

In [204]:
MFW = titles_article(soup)
MFW

['Giorgio Armani',
 'Bottega Veneta',
 'Dolce & Gabbana',
 'MSGM',
 'Ferragamo',
 'Ferrari',
 'Jil Sander',
 'Andreadamo',
 'Gucci',
 'Sportmax',
 'Tod’s',
 'Blumarine',
 'Emporio Armani',
 'Prada',
 'Max Mara',
 'Etro',
 'No. 21',
 'Fendi',
 'Diesel']

         Paris Fashion week info extraction. Participating brands.

In [205]:
''' Reusing the function by calling it'''

url_3 = "https://www.fhcm.paris/en/paris-fashion-week/calendar"
soup = download_html(url_3)

In [206]:
ds =  soup.find_all("div", {"class": "cal-item shown"})
a =[i.find("a") for i in ds]
names = [i.find("h3") for i in a]
brands_filtered = [i.getText().title() for i in names]
brands_filtered

['Marie Adam-Leenaerdt',
 'Weinsanto',
 'Benjamin Benmoyal',
 'Vaquera',
 'Mossi',
 'Pierre Cardin',
 'Barbara Bui',
 'Peter Do',
 'Niccolò Pasqualetti',
 'Cfcl',
 'Victoria/Tomas',
 'Mame Kurogouchi',
 'Christian Dior',
 'Dawei',
 'Florentina Leitner',
 'Anrealage',
 'Pressiat',
 'Germanier',
 'Heliot Emil',
 'Saint Laurent',
 'Courreges',
 'Reverie By Caroline Hù',
 'The Row',
 'Marni',
 'Minuit',
 'Dries Van Noten',
 'Kimhekim',
 'Cecilie Bahnsen',
 'Litkovska',
 'Undercover',
 'Didu',
 'Acne Studios',
 'Balmain',
 'Uma Wang',
 'Veronique Leroy',
 'Lanvin',
 'Rabanne',
 'Gauchere',
 'Chloé',
 'Givenchy',
 'Nehera',
 'Christopher Esber',
 'Rick Owens',
 'Schiaparelli',
 'Isabel Marant',
 'Alexis Mabille',
 'Alexandre Vauthier',
 'Leonard Paris',
 'Loewe',
 'Maitrepierre',
 'Issey Miyake',
 'Jitrois',
 'Giambattista Valli',
 'Lutz Huelle',
 'Nina Ricci',
 'Enfants Riches Deprimes',
 'Victoria Beckham',
 'Situationist',
 'Yohji Yamamoto',
 'Coperni',
 'Junya Watanabe',
 'Carven',
 'Rui

    ''' Now let's encapsulate'''

In [207]:
''' Get brands from containers in a calendar in html. '''

def brands_filtered(soup):
    ds =  soup.find_all("div", {"class": "cal-item shown"})
    a =[i.find("a") for i in ds]
    names = [i.find("h3") for i in a]
    return [i.getText().title() for i in names]

In [208]:
''' Calling the functions'''

PFW = brands_filtered(soup)
PFW

['Marie Adam-Leenaerdt',
 'Weinsanto',
 'Benjamin Benmoyal',
 'Vaquera',
 'Mossi',
 'Pierre Cardin',
 'Barbara Bui',
 'Peter Do',
 'Niccolò Pasqualetti',
 'Cfcl',
 'Victoria/Tomas',
 'Mame Kurogouchi',
 'Christian Dior',
 'Dawei',
 'Florentina Leitner',
 'Anrealage',
 'Pressiat',
 'Germanier',
 'Heliot Emil',
 'Saint Laurent',
 'Courreges',
 'Reverie By Caroline Hù',
 'The Row',
 'Marni',
 'Minuit',
 'Dries Van Noten',
 'Kimhekim',
 'Cecilie Bahnsen',
 'Litkovska',
 'Undercover',
 'Didu',
 'Acne Studios',
 'Balmain',
 'Uma Wang',
 'Veronique Leroy',
 'Lanvin',
 'Rabanne',
 'Gauchere',
 'Chloé',
 'Givenchy',
 'Nehera',
 'Christopher Esber',
 'Rick Owens',
 'Schiaparelli',
 'Isabel Marant',
 'Alexis Mabille',
 'Alexandre Vauthier',
 'Leonard Paris',
 'Loewe',
 'Maitrepierre',
 'Issey Miyake',
 'Jitrois',
 'Giambattista Valli',
 'Lutz Huelle',
 'Nina Ricci',
 'Enfants Riches Deprimes',
 'Victoria Beckham',
 'Situationist',
 'Yohji Yamamoto',
 'Coperni',
 'Junya Watanabe',
 'Carven',
 'Rui

         New York week info extraction. Participating brands.

In [209]:
''' Reusing the function by calling it'''

url_4 = "https://www.vogue.com/fashion-shows"
soup = download_html(url_4)

In [210]:
''' I tried different ways to get the information I needed because at first it gave me a hole lot more
than I needed. This is the final way that worked. '''

#ds =  soup.find_all("nav", {"class": "NavigationWrapper-bFftAs fKSZRK GroupedNavigationGroup-KHsBC cKGNRO grouped-navigation__group"})
li = soup.find_all("li", {"class":"NavigationListItemWrapper-cxLZKD iAYgTw link--secondary navigation__list-item"})
names = [i.getText() for i in li]
names
#a = [i.find("a", {"class":"NavigationInternalLink-cWEaeo knabMb grouped-navigation__link link--secondary navigation__link"}) for i in li]
#a


['A. Potts',
 'A.L.C.',
 'A.P.C.',
 'A.W.A.K.E. Mode',
 'Acne Studios',
 'Adam Lippes',
 'Adeam',
 'AG',
 'AGL',
 'Ahluwalia',
 'Akris',
 'Alaïa',
 'Alberta Ferretti',
 'Alejandra Alonso Rojas',
 'Alessandra Rich',
 'Alexander McQueen',
 'Alexandre Vauthier',
 'Alexis Mabille',
 'Alice + Olivia',
 'All-In',
 'Altuzarra',
 'Ambush',
 'Andreadamo',
 'Andreas Kronthaler for Vivienne Westwood',
 'Ann Demeulemeester',
 'Anna October',
 'Anna Sui',
 'Anrealage',
 'Anteprima',
 'Antonio Marras',
 'Apiece Apart',
 'Armarium',
 'Arthur Arbesser',
 'ASAI',
 'Ashish',
 'Ashley Williams',
 'Ashlyn',
 'Atlein',
 'ATXV',
 'Avavav',
 'Axel Arigato',
 'AZ Factory',
 'Bach Mai',
 'Badgley Mischka',
 'Balenciaga',
 'Bally',
 'Balmain',
 'Barbara Tfank',
 'Batsheva',
 'Bevza',
 'Bibhu Mohapatra',
 'Bite Studios',
 'Blumarine',
 'Bora Aksu',
 'Bottega Veneta',
 'Brandon Maxwell',
 'Bronx and Banco',
 'Brunello Cucinelli',
 'Burberry',
 'By Malene Birger',
 'Câllas Milano',
 'Calvin Luo',
 'Carolina Herrer

    ''' Now let's encapsulate. '''

In [211]:
''' Function to get the brand names from a nav bar'''

def get_nav_brands(soup):
    li = soup.find_all("li", {"class":"NavigationListItemWrapper-cxLZKD iAYgTw link--secondary navigation__list-item"})
    return [i.getText() for i in li]

In [212]:
''' Calling the function and assigning the name of New York Fashion Week. '''

NYFW = get_nav_brands(soup)
NYFW

['A. Potts',
 'A.L.C.',
 'A.P.C.',
 'A.W.A.K.E. Mode',
 'Acne Studios',
 'Adam Lippes',
 'Adeam',
 'AG',
 'AGL',
 'Ahluwalia',
 'Akris',
 'Alaïa',
 'Alberta Ferretti',
 'Alejandra Alonso Rojas',
 'Alessandra Rich',
 'Alexander McQueen',
 'Alexandre Vauthier',
 'Alexis Mabille',
 'Alice + Olivia',
 'All-In',
 'Altuzarra',
 'Ambush',
 'Andreadamo',
 'Andreas Kronthaler for Vivienne Westwood',
 'Ann Demeulemeester',
 'Anna October',
 'Anna Sui',
 'Anrealage',
 'Anteprima',
 'Antonio Marras',
 'Apiece Apart',
 'Armarium',
 'Arthur Arbesser',
 'ASAI',
 'Ashish',
 'Ashley Williams',
 'Ashlyn',
 'Atlein',
 'ATXV',
 'Avavav',
 'Axel Arigato',
 'AZ Factory',
 'Bach Mai',
 'Badgley Mischka',
 'Balenciaga',
 'Bally',
 'Balmain',
 'Barbara Tfank',
 'Batsheva',
 'Bevza',
 'Bibhu Mohapatra',
 'Bite Studios',
 'Blumarine',
 'Bora Aksu',
 'Bottega Veneta',
 'Brandon Maxwell',
 'Bronx and Banco',
 'Brunello Cucinelli',
 'Burberry',
 'By Malene Birger',
 'Câllas Milano',
 'Calvin Luo',
 'Carolina Herrer

''' The plan with the lists of participating brands on the 3 Fashion Weeks is to create 3 columns with the fashion weeks names and compare the lists with the brands in the base dataframe and assign to these new columns a boolean value if the brand participated in that specific fashion Week. '''

6. Get a list of Designers of the brands from the base dataset. And if possible the year of "icorporación".

In [213]:
''' Reusing the function by calling it'''

url_5 = "https://highxtar.com/la-cfda-anuncia-los-nominados-para-el-2020/"
soup = download_html(url_5)

In [214]:
''' Function to get list of Designers. This one was harsher as every needed item was a "br" inside a "p"
so it wasn't possible to extract specifically the item. But founnf a workaround by indexing.'''

def get_designers(soup):
    ds = soup.find("div", {"class":"content__main__content"})
    p = ds.find_all("p")
    text = [i.getText() for i in p]
    lists = [i.split("\n") for i in text if "CFDA" not in i]
    return [i[1:] for i in lists]

In [215]:
designers = get_designers(soup)
designers

[['', '', '', ''],
 ['Mary-Kate and Ashley Olsen, The Row',
  'Brandon Maxwell',
  'Gabriela Hearst',
  'Marc Jacobs',
  'Tom Ford'],
 ['Emily Adams Bode, Bode',
  'Kerby Jean-Raymond, Pyer Moss',
  'Todd Snyder',
  'Thom Browne',
  'Tom Ford'],
 ['Mary-Kate and Ashley Olsen, The Row',
  'Gabriela Hearst',
  'Stuart Vevers, Coach',
  'Jennifer Fisher, Jennifer Fisher Jewelry',
  'Telfar Clemens, Telfar'],
 ['Christopher John Rogers',
  'Kenneth Nicholson',
  'Peter Do',
  'Reese Cooper',
  'Sarah Staudinger and George Augusto, STAUD'],
 ['Daniel Lee, Bottega Veneta',
  'Dries Van Noten',
  'Miuccia Prada, Prada',
  'Pierpaolo Piccioli, Valentino',
  'Rick Owens'],
 ['Craig Green',
  'Dries Van Noten',
  'Jonathan Anderson, Loewe',
  'Kim Jones, Dior',
  'Virgil Abloh, Louis Vuitton']]